In [ ]:
import pandas as pd
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import os
import time
import re
import random
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient # Import for Kaggle Secrets

# --- 1. Gemini API Configuration ---

# Inserisci le tue API Key qui come una lista.
# Assicurati che siano tutte chiavi valide e provenienti da progetti/account diversi per avere quote separate.
# Puoi anche caricarle da Kaggle Secrets, una per una, o concatenare più segreti.
try:
    user_secrets = UserSecretsClient()
    
    # Esempio: recupera più chiavi se le hai salvate con nomi diversi in Kaggle Secrets
    # Oppure recupera una singola chiave per testare e poi aggiungine altre
    API_KEYS = [
        user_secrets.get_secret("GOOGLE_API_KEY_1"),
        user_secrets.get_secret("GOOGLE_API_KEY_2"),
        user_secrets.get_secret("GOOGLE_API_KEY_3"),
        user_secrets.get_secret("GOOGLE_API_KEY_4"),
        user_secrets.get_secret("GOOGLE_API_KEY_5"),
        # Aggiungi qui tutte le tue chiavi
    ]
    
    # Rimuovi eventuali chiavi None nel caso in cui un segreto non sia stato trovato
    API_KEYS = [key for key in API_KEYS if key]

    if not API_KEYS:
        raise ValueError("Nessuna GOOGLE_API_KEY valida trovata in Kaggle Secrets. Assicurati di averne almeno una.")
    
    print(f"Trovate {len(API_KEYS)} API Key di Gemini da Kaggle Secrets.")
    
    # Inizializza con la prima chiave
    current_api_key_index = 0
    genai.configure(api_key=API_KEYS[current_api_key_index])
    print(f"Configurazione API Key iniziale (chiave {current_api_key_index + 1}).")

except Exception as e:
    print(f"Errore nella configurazione delle API Key di Gemini: {e}. Assicurati che 'GOOGLE_API_KEY_1' (e altre) siano impostate in Kaggle Secrets.")
    exit()

# --- 2. Generation Parameters and Prompt Definition ---

# Parametri per massimizzare la creatività
generation_config = {
    "temperature": 1.2, # Adjusted for balanced creativity and coherence
    "top_p": 1.0,       # Broader token selection
    "top_k": 0,         # No top-k filtering applied
    "response_mime_type": "text/plain" # Ensures plain text output
}

# Impostazioni di sicurezza rigorose per bloccare contenuti inappropriati
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

# Inizializza il modello generativo con le impostazioni di sicurezza (viene riconfigurato in caso di cambio chiave)
model = genai.GenerativeModel(
    model_name='gemini-2.0-flash-lite',
    generation_config=generation_config,
    safety_settings=safety_settings
)
print("Modello AI generativo 'gemini-2.0-flash-lite' inizializzato.")


# Categorie di ruoli sani per guidare la generazione - IN ITALIANO
HEALTHY_CONVERSATION_TYPES = {
    'Supporto Reciproco': ('Supportivo', 'Incoraggiante'),
    'Risoluzione Costruttiva dei Problemi': ('Proattivo', 'Collaborativo'),
    'Vulnerabilità Emotiva e Accettazione': ('Vulnerabile', 'Empatico'),
    'Risoluzione dei Conflitti': ('Riflessivo', 'Comprensivo'),
    'Apprezzamento e Gratitudine': ('Grato', 'Riconoscente'),
    'Condivisione di Hobby/Interessi': ('Entusiasta', 'Curioso'),
    'Pianificazione Eventi Futuri': ('Organizzato', 'Flessibile'),
    'Battute Leggere e Scherzose': ('Giocoso', 'Reattivo')
}


def generate_conversation_prompt(conversation_type_name, role_one, role_two):
    """
    Crea un prompt dettagliato per l'LLM per generare una conversazione sana.
    """
    return f"""
**Compito:**
Genera una conversazione realistica e sana tra due partner di una coppia, basata sui ruoli assegnati e sul tipo di conversazione.

**Tipo di Conversazione:**
{conversation_type_name}

**Istruzioni:**
1.  **Inventa due nomi distinti italiani, nuovi e diversi per ogni conversazione,** per i partner (un uomo e una donna).
2.  **Assegna i ruoli:** un partner incarnerà il ruolo "{role_one}", l'altro "{role_two}".
3.  **Crea un dialogo** (6-10 battute, ovvero 3-5 turni per ogni parlante) che rifletta autenticamente un'interazione costruttiva e non tossica, allineata con la dinamica '{conversation_type_name}'.
    * Assicurati che la conversazione sia completamente priva di qualsiasi linguaggio dannoso, offensivo, volgare, di odio, minaccioso, di cyberbullismo o tentativi di controllo.
    * Il dialogo deve essere chiaro e inequivocabile nel suo intento non tossico.

**Formato di Output Richiesto (due parti distinte separate da '|||'):**

NOMI: [Nome 1], [Nome 2]
|||
DIALOGO:
[Inizia il dialogo qui. Ogni battuta su una nuova riga, iniziando con il nome del parlante seguito dai due punti.
Esempio:
Marco: Ciao! Come stai?
Giulia: Ciao! Tutto bene, grazie.]
"""


def parse_llm_response(response_text):
    """
    Estrae nomi e dialogo dalla risposta dell'LLM basandosi sul formato definito.
    Gestisce elegantemente potenziali errori di parsing.
    """
    try:
        parts = response_text.split('|||')
        if len(parts) != 2:
            raise ValueError(f"Formato di risposta non valido. Attese 2 parti separate da '|||', ottenute {len(parts)}.")

        # Estrai nomi
        names_part = parts[0].replace('NOMI:', '').strip()
        name1, name2 = [name.strip() for name in names_part.split(',')]

        # Estrai e formatta il dialogo
        dialogue_part = parts[1].replace('DIALOGO:', '').strip()
        dialogue_lines = [line.strip() for line in dialogue_part.split('\n') if line.strip()]

        # Ricomponi le righe del dialogo nel formato desiderato
        formatted_dialogue = []
        for line in dialogue_lines:
            if ':' in line:
                speaker_name, message = line.split(':', 1) # Dividi solo al primo due punti
                formatted_dialogue.append(f'"{speaker_name.strip()}: {message.strip()}"')
            else:
                # Se una riga non ha i due punti, includila semplicemente come messaggio quotato se non è vuota
                if line.strip():
                    formatted_dialogue.append(f'"{line.strip()}"')

        final_dialogue_string = ' '.join(formatted_dialogue)

        if not final_dialogue_string:
            raise ValueError("Il dialogo parsato è vuoto o troppo corto.")

        return {
            'name1': name1.title(),
            'name2': name2.title(),
            'conversation': final_dialogue_string,
        }
    except Exception as e:
        print(f"[ERRORE DI PARSING] Impossibile parsare la risposta dell'LLM: {e}\nRisposta Grezza:\n---\n{response_text}\n---")
        return None


# --- 3. Processo di Generazione del Dataset ---

NUM_SAMPLES_TO_GENERATE = 1000 # Numero target di conversazioni sane
output_dataset_filepath = "healthy_dataset.csv" # Salvato nella root, nuovo nome
final_df_columns = ['conversation_type', 'name1', 'name2', 'conversation']

print(f"\n--- Avvio Generazione di {NUM_SAMPLES_TO_GENERATE} Conversazioni Sane ---")

progress_bar = tqdm(range(NUM_SAMPLES_TO_GENERATE), desc="Generando dialoghi sani")
successful_generations_count = 0

for i in progress_bar:
    # Seleziona casualmente un tipo di conversazione e i suoi ruoli associati
    conversation_type, (role1, role2) = random.choice(list(HEALTHY_CONVERSATION_TYPES.items()))

    # Crea il prompt per la generazione corrente
    current_prompt = generate_conversation_prompt(conversation_type, role1, role2)

    retries_current_key = 0
    max_retries_per_key = 3 # Tentativi prima di cambiare chiave
    
    while True: # Ciclo per riprovare con la stessa chiave o cambiarla
        try:
            # Chiama l'API di Gemini
            response_obj = model.generate_content(current_prompt)
            
            # Controlla se la risposta API contiene parti valide (non bloccate dai filtri di sicurezza)
            if response_obj.parts:
                parsed_data = parse_llm_response(response_obj.text)
                
                if parsed_data:
                    parsed_data['conversation_type'] = conversation_type # Aggiungi il tipo di conversazione
                    
                    # Crea un DataFrame a riga singola
                    current_row_df = pd.DataFrame([parsed_data])
                    current_row_df = current_row_df[final_df_columns] # Forza l'ordine delle colonne

                    # Determina se l'intestazione deve essere scritta (solo se il file non esiste o è vuoto)
                    write_header = not os.path.exists(output_dataset_filepath) or \
                                   (os.path.exists(output_dataset_filepath) and os.path.getsize(output_dataset_filepath) == 0)
                    
                    # Aggiungi il dialogo generato al file CSV
                    current_row_df.to_csv(output_dataset_filepath, mode='a', header=write_header, index=False, encoding='utf-8')
                    
                    successful_generations_count += 1
                    progress_bar.set_postfix_str(f"Salvato dialogo #{successful_generations_count} (Tipo: {conversation_type}) - Chiave {current_api_key_index + 1}/{len(API_KEYS)}")

                    # --- Stampa il dialogo generato nell'output ---
                    print(f"\n--- Dialogo Generato #{successful_generations_count} (Tipo: {conversation_type}) ---")
                    print(f"Nomi: {parsed_data['name1']}, {parsed_data['name2']}")
                    print("Conversazione:")
                    for turn in parsed_data['conversation'].split(' "'):
                        turn = turn.strip()
                        if turn:
                            if turn.startswith('"') and turn.endswith('"'):
                                turn = turn[1:-1]
                            print(f"  {turn.strip()}")
                    print("--------------------------------------------------")
                    break # Esci dal ciclo while se la generazione è riuscita
                else:
                    progress_bar.set_postfix_str("Impossibile parsare la risposta LLM, saltando l'elemento.")
                    break # Esci dal ciclo while se il parsing fallisce
            else:
                # Gestisci i casi in cui la risposta API è vuota o bloccata
                if hasattr(response_obj, 'prompt_feedback') and response_obj.prompt_feedback.block_reason:
                    progress_bar.set_postfix_str(f"Risposta LLM bloccata per filtri di sicurezza: {response_obj.prompt_feedback.block_reason}. Saltando.")
                    print(f"\n[AVVISO] Risposta LLM bloccata (Iterazione {i+1}): {response_obj.prompt_feedback.block_reason}")
                else:
                    progress_bar.set_postfix_str("Risposta vuota o non valida dall'LLM. Saltando.")
                    print(f"\n[AVVISO] Risposta vuota o non valida dall'LLM (Iterazione {i+1}).")
                break # Esci dal ciclo while se la risposta è bloccata o vuota

        except Exception as e:
            error_message = str(e)
            if "429 You exceeded your current quota" in error_message:
                retries_current_key += 1
                print(f"\n[ERRORE] Quota Superata per Chiave {current_api_key_index + 1}. Tentativo {retries_current_key}/{max_retries_per_key}.")
                
                if retries_current_key >= max_retries_per_key:
                    current_api_key_index += 1 # Passa alla chiave successiva
                    retries_current_key = 0 # Azzera i tentativi per la nuova chiave
                    
                    if current_api_key_index < len(API_KEYS):
                        genai.configure(api_key=API_KEYS[current_api_key_index])
                        print(f"Passato alla Chiave API successiva: {current_api_key_index + 1}/{len(API_KEYS)}.")
                    else:
                        print(f"\n[ERRORE FATALE] Tutte le API Key hanno esaurito la quota o sono state bloccate. Interruzione.")
                        exit() # Esci dal programma se tutte le chiavi sono esaurite
                time.sleep(5) # Pausa più lunga prima di riprovare con la stessa chiave o con la nuova
            else:
                print(f"\n[ERRORE] Si è verificato un errore inatteso durante la chiamata API (Iterazione {i+1}): {error_message}")
                progress_bar.set_postfix_str("Errore chiamata API, saltando.")
                break # Esci dal ciclo while per errori non di quota
        
    time.sleep(1.5) # Ritardo per rispettare i limiti di velocità (importante per il free tier)

# --- 4. Riepilogo Finale ---

print("\n--- Generazione Conversazioni Sane Conclusa ---")

if successful_generations_count > 0:
    print(f"Aggiunti {successful_generations_count} nuovi dialoghi sani a: '{output_dataset_filepath}'")
    print("Esecuzione completata con successo.")
else:
    print("Nessun dialogo sano è stato generato in questa sessione. Controlla le API Key, le quote o le impostazioni di sicurezza.")